In [2]:
from google.colab import drive

print("--> Attempting to force remount Google Drive...")
drive.mount('/content/drive', force_remount=True)
print("--> Google Drive mounted successfully!")

--> Attempting to force remount Google Drive...
Mounted at /content/drive
--> Google Drive mounted successfully!


In [9]:
# Block to re-run: Paste your CORRECT path and execute.

import pandas as pd
import numpy as np
import os
import re
import gc
from scipy.sparse import hstack, csr_matrix, save_npz
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from io import BytesIO
import requests
from tqdm import tqdm

# --- STEP 0: DEFINE ALL FUNCTIONS AND MODELS ---
print("Step 0: Defining all necessary functions and loading models...")
def extract_pack_quantity(text):
    if not isinstance(text, str): return 1.0
    match = re.search(r'Value:\s*([\d.]+)', text)
    return float(match.group(1)) if match else 1.0

def get_image_features(image_path, model):
    try:
        img = Image.open(image_path).convert('RGB')
        img_t = preprocess(img)
        batch_t = torch.unsqueeze(img_t, 0)
        with torch.no_grad(): features = model(batch_t)
        return features.squeeze().numpy()
    except Exception as e:
        return np.zeros(2048)

vectorizer = TfidfVectorizer(max_features=20000, stop_words='english', ngram_range=(1, 2))
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model = torch.nn.Sequential(*(list(model.children())[:-1]))
model.eval()
preprocess = transforms.Compose([
    transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# --- STEP 1: PROCESS IN CHUNKS AND SAVE TO GOOGLE DRIVE ---
print("\nStep 1: Starting the chunking process...")
CHUNK_SIZE = 2500
# !!! PASTE THE PATH YOU COPIED FROM THE FILE EXPLORER HERE !!!
TRAIN_FILE = '/content/drive/MyDrive/ML DATASET/student_resource/dataset/train.csv'
IMAGE_DIR = '/content/drive/MyDrive/ML_Competition/full_train_images'
FEATURES_DIR = '/content/drive/MyDrive/ML_Competition/processed_features'

os.makedirs(FEATURES_DIR, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)

try:
    existing_chunks = len(os.listdir(FEATURES_DIR))
    if existing_chunks > 0:
        print(f"--> RESUMING from after chunk {existing_chunks}. Skipping {existing_chunks * CHUNKS_SIZE} rows.")
        print("--> Fitting vectorizer on first chunk to ensure consistency...")
        temp_df = pd.read_csv(TRAIN_FILE, nrows=CHUNK_SIZE)
        temp_df['catalog_content'] = temp_df['catalog_content'].fillna('')
        vectorizer.fit(temp_df['catalog_content'])
        del temp_df

        skip_rows = existing_chunks * CHUNKS_SIZE
        chunk_iterator = pd.read_csv(TRAIN_FILE, chunksize=CHUNK_SIZE, skiprows=range(1, skip_rows + 1))
    else:
        chunk_iterator = pd.read_csv(TRAIN_FILE, chunksize=CHUNK_SIZE)


    for i, df_chunk in enumerate(chunk_iterator, start=existing_chunks):
        print(f"\n--- Processing Chunk {i+1} ---")

        if i == 0 and existing_chunks == 0:
            print("--> Fitting TF-IDF on the first chunk...")
            df_chunk['catalog_content'] = df_chunk['catalog_content'].fillna('')
            vectorizer.fit(df_chunk['catalog_content'])

        df_chunk['price'] = pd.to_numeric(df_chunk['price'], errors='coerce')
        df_chunk.dropna(subset=['price'], inplace=True)
        df_chunk['log_price'] = np.log1p(df_chunk['price'])
        df_chunk['item_pack_quantity'] = df_chunk['catalog_content'].apply(extract_pack_quantity)
        df_chunk['catalog_content'] = df_chunk['catalog_content'].fillna('')
        df_chunk['item_pack_quantity'] = df_chunk['item_pack_quantity'].fillna(1)
        text_features = vectorizer.transform(df_chunk['catalog_content'])
        engineered_features = csr_matrix(df_chunk[['item_pack_quantity']].values)

        print(f"--> Downloading/Verifying {len(df_chunk)} images for chunk {i+1}...")
        for _, row in tqdm(df_chunk.iterrows(), total=df_chunk.shape[0]):
            image_path = os.path.join(IMAGE_DIR, f"{row['sample_id']}.jpg")
            if not os.path.exists(image_path):
                try:
                    response = requests.get(row['image_link'], timeout=20)
                    if response.status_code == 200:
                        Image.open(BytesIO(response.content)).convert('RGB').save(image_path)
                except Exception: pass

        print(f"--> Extracting image features for chunk {i+1}...")
        image_features_list = [get_image_features(os.path.join(IMAGE_DIR, f"{sid}.jpg"), model) for sid in tqdm(df_chunk['sample_id'])]
        image_features_chunk = np.array(image_features_list)
        X_chunk = hstack([text_features, engineered_features, csr_matrix(image_features_chunk)])
        y_chunk = df_chunk['log_price']

        chunk_filename = os.path.join(FEATURES_DIR, f"features_chunk_{i}.npz")
        np.savez_compressed(chunk_filename, data=X_chunk.data, indices=X_chunk.indices,
                 indptr=X_chunk.indptr, shape=X_chunk.shape, labels=y_chunk.values)
        print(f"--> SUCCESS: Saved chunk {i+1} to {chunk_filename}")

        del df_chunk, text_features, engineered_features, image_features_chunk, X_chunk, y_chunk
        gc.collect()

    print("\n\n*** FEATURE EXTRACTION COMPLETE! ***")

except FileNotFoundError:
    print(f"CRITICAL ERROR: The file '{TRAIN_FILE}' was not found. Please double-check the path.")
except Exception as e:
    print(f"AN UNEXPECTED ERROR OCCURRED: {e}")

Step 0: Defining all necessary functions and loading models...

Step 1: Starting the chunking process...

--- Processing Chunk 1 ---
--> Fitting TF-IDF on the first chunk...
--> Downloading/Verifying 2500 images for chunk 1...


100%|██████████| 2500/2500 [05:21<00:00,  7.77it/s]


--> Extracting image features for chunk 1...


100%|██████████| 2500/2500 [11:54<00:00,  3.50it/s]


--> SUCCESS: Saved chunk 1 to /content/drive/MyDrive/ML_Competition/processed_features/features_chunk_0.npz

--- Processing Chunk 2 ---
--> Downloading/Verifying 2500 images for chunk 2...


...

--> SUCCESS: Saved chunk 29 to /content/drive/MyDrive/ML_Competition/processed_features/features_chunk_28.npz

--- Processing Chunk 30 ---
--> Downloading/Verifying 2500 images for chunk 30...


100%|██████████| 2500/2500 [05:20<00:00,  7.81it/s]


--> Extracting image features for chunk 30...


100%|██████████| 2500/2500 [12:16<00:00,  3.39it/s]


--> SUCCESS: Saved chunk 30 to /content/drive/MyDrive/ML_Competition/processed_features/features_chunk_29.npz


*** FEATURE EXTRACTION COMPLETE! ***
